# Imports

In [13]:
import numpy as np
import matplotlib.pyplot as plt
import tensorflow.keras as keras
import os
import cv2
from sklearn.model_selection import train_test_split
import xml.etree.ElementTree as ET
import glob
from tensorflow.keras.models import Sequential
from tensorflow.keras import layers

# Get train images of weapons

In [2]:
if 'BaggagesData' is not os.getcwd():
    os.chdir('../BaggagesData')

folderPath = '../BaggagesData'
dim = (800,800)

x_train = []
for folder in os.listdir(folderPath):
    if 'B00' in folder:
        for file in os.listdir(folderPath+"/"+folder):
            temp = cv2.imread(os.path.join(folderPath+"/"+folder,file))
            if temp is not None:
                temp = cv2.resize(temp, dim, interpolation=cv2.INTER_CUBIC)
                x_train.append(temp)

#x_train = np.array(x_train)
#print(x_train.shape)

# Get test images of weapons

In [3]:
x_test = []
for file in os.listdir(folderPath+"/Test"):
    temp = cv2.imread(os.path.join(folderPath+"/Test",file))
    if temp is not None:
        temp = cv2.resize(temp, dim, interpolation=cv2.INTER_CUBIC)
        x_test.append(temp)
                
#x_test = np.array(x_test)
#n_test = len(x_test)
#print(x_test.shape)

# Get images with no weapons

In [4]:
no_wep = []
for file in os.listdir(folderPath+"/no_wep"):
    temp = cv2.imread(os.path.join(folderPath+"/no_wep",file))
    if temp is not None:
        temp = cv2.resize(temp, dim, interpolation=cv2.INTER_CUBIC)
        #temp = cv2.cvtColor(temp, cv2.COLOR_BGR2GRAY)
        no_wep.append(temp)

#n_no_wep = len(no_wep)
#no_wep = np.array(no_wep)
#print(no_wep.shape)

# Add no_weapon data to x_train and x_test

In [5]:
x_train.extend(no_wep[:12])
x_test.extend(no_wep[12:])

In [6]:
x_train = np.array(x_train)
x_test = np.array(x_test)

print(len(x_test), len(x_train))

[[[238 238 238]
  [237 237 237]
  [235 235 235]
  ...
  [240 240 240]
  [240 240 240]
  [237 237 237]]

 [[240 240 240]
  [238 238 238]
  [238 238 238]
  ...
  [239 239 239]
  [238 238 238]
  [235 235 235]]

 [[238 238 238]
  [237 237 237]
  [236 236 236]
  ...
  [233 233 233]
  [232 232 232]
  [235 235 235]]

 ...

 [[234 234 234]
  [236 236 236]
  [237 237 237]
  ...
  [238 238 238]
  [238 238 238]
  [238 238 238]]

 [[234 234 234]
  [236 236 236]
  [237 237 237]
  ...
  [238 238 238]
  [238 238 238]
  [238 238 238]]

 [[236 236 236]
  [236 236 236]
  [237 237 237]
  ...
  [238 238 238]
  [237 237 237]
  [237 237 237]]]
65 958


# Create categorical labels

In [7]:
n_categories = 2
train_wep = 12
test_wep = 4

print(x_train.shape, x_test.shape)
y_train = np.ones(len(x_train))
y_train[-train_wep:] = 2
y_train = keras.utils.to_categorical(y_train-1, n_categories)

y_test = np.ones(len(x_test))
y_test[-test_wep:] = 2
y_test = keras.utils.to_categorical(y_test-1, n_categories)

(958, 800, 800, 3) (65, 800, 800, 3)


# Train the model

In [18]:
n, y, x, d = x_train.shape
inputShape = (y,x,d)
n_iterations = 3

model = Sequential()
model.add(layers.Conv2D(8, (2, 2), activation='relu', input_shape=inputShape))
model.add(layers.MaxPooling2D(pool_size=(2, 2)))
model.add(layers.Conv2D(4, (2, 2), activation='relu'))
model.add(layers.MaxPooling2D(pool_size=(2, 2)))
#model.add(layers.Conv2D(2, (2, 2), activation='relu'))
model.add(layers.Flatten())
model.add(layers.Dense(8, activation='relu'))
model.add(layers.Dense(units = n_categories, activation='softmax'))

# compiling model with cross entropy and accuracy
model.compile(loss='categorical_crossentropy', metrics=['accuracy', keras.metrics.Precision(), keras.metrics.Recall()])

# training model
model.fit(x_train, y_train, epochs=n_iterations, verbose=1, validation_data=(x_test, y_test))

Epoch 1/3
30/30 [==============================] - 53s 2s/step - loss: 13.1087 - accuracy: 0.9666 - precision_4: 0.9666 - recall_4: 0.9666 - val_loss: 11.4042 - val_accuracy: 0.9385 - val_precision_4: 0.9385 - val_recall_4: 0.9385
Epoch 2/3
30/30 [==============================] - 54s 2s/step - loss: 0.9858 - accuracy: 0.9395 - precision_4: 0.9395 - recall_4: 0.9395 - val_loss: 0.6719 - val_accuracy: 0.9385 - val_precision_4: 0.9385 - val_recall_4: 0.9385
Epoch 3/3
30/30 [==============================] - 53s 2s/step - loss: 0.6556 - accuracy: 0.9875 - precision_4: 0.9875 - recall_4: 0.9875 - val_loss: 0.6465 - val_accuracy: 0.9385 - val_precision_4: 0.9385 - val_recall_4: 0.9385


# Testing the model

In [21]:
loss, accuracy, precision, recall = model.evaluate(x_test, y_test, verbose=0)

print("Loss:\t\t {:.4f}".format(loss))
print("Accuracy:\t {:.4f}".format(accuracy))
print("Precision:\t {:.4f}".format(precision))
print("Recall:\t\t {:.4f}".format(recall))

Loss:		 0.6465
Accuracy:	 0.9385
Precision:	 0.9385
Recall:		 0.9385
